In [2]:
import numpy as np
import pandas as pd
import cv2
import PIL
import pytesseract
# import keras_ocr
# import tensorflow
import spacy
import easyocr
import time

In [2]:
def resizer(image,width=800):
    # get widht and height
    h,w,c = image.shape 
    height = int((h/w)* width )
    size = (width,height)
    image = cv2.resize(image,(width,height))
    return image, size 

In [6]:
img_cv = cv2.imread('./invoices/1.png') 
img_pytesseract_ = img_cv.copy()
img_pytesseract, size = resizer(img_pytesseract_)
cv2.imshow('invoice',img_pytesseract)
cv2.waitKey(0)
cv2.destroyAllWindows()
start_time = time.time()
text_cv = pytesseract.image_to_string(img_pytesseract)
print(format(time.time() - start_time))
print(text_cv)

0.5477674007415771
Company Name INVOICE

[Street Address]
city, ST ZIP] pate | 12/9/2019
INVOICE #
CUSTOMER ID
DUE DATE

[City, ST ZIP]
[Phone]
DESCRIPTION TAXED AMOUNT
[Service Fee] 230.00
[Labor: 5 hours at $75/hr] 375.00
[Parts] x 345.00
Subtotal 950.00
Taxable 345.00
Tax rate 6.2508|
1, Total payment due in 30 days Tax due 21.56
2. Please include the invoice number on your check Other
S$ 971.56
Make all checks payable to
[Your Company Name]

If you have any questions about this invoice, please contact
[Name, Phone #, E-mail]
Thank You For Your Business!

https:/werw.vertex42,comvExce!Templates excel-invoice-template html Invoice Template ® 2010-2019 by Vertex42.com




In [7]:
data = pytesseract.image_to_data(img_pytesseract)
dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
df = pd.DataFrame(dataList[1:],columns=dataList[0])
df.dropna(inplace=True)
df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,800,1035,-1,
1,2,1,1,0,0,0,0,0,800,3,-1,
2,3,1,1,1,0,0,0,0,800,3,-1,
3,4,1,1,1,1,0,0,0,800,3,-1,
4,5,1,1,1,1,1,0,0,800,3,95.000000,
...,...,...,...,...,...,...,...,...,...,...,...,...
176,5,1,15,1,1,1,2,1032,798,3,95.000000,
177,2,1,16,0,0,0,797,2,3,1033,-1,
178,3,1,16,1,0,0,797,2,3,1033,-1,
179,4,1,16,1,1,0,797,2,3,1033,-1,


In [8]:

col_int = ['level','page_num','block_num','par_num','line_num','word_num','left','top','width','height']
df[col_int] = df[col_int].astype(object).astype(int)

image = img_pytesseract
level = 'word'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
#     print(l,x,y,w,h,c,txt)    
    if level == 'page':
        if l == 1:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,0),1)
        else:
            continue
            
    elif level == 'block':
        if l == 2:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),1)
        else:
            continue
    
    elif level == 'para':
        if l == 3:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),1)
        else:
            continue
    
    elif level == 'line':
        if l == 4:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),1)
        else:
            continue
            
    elif level == 'word':
        if l == 5:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),1)
          
        else:
            continue
            
cv2.imshow("invoice pytesseract",image)
cv2.waitKey()
cv2.destroyAllWindows()